In [1]:
!pwd

/home/nlp/preprocess


In [2]:
import os
import re
from tqdm import tqdm

In [3]:
# DATA PATH

data_path = "/home/nlp/preprocess/datas/"
data_name = "namuwiki-210301-part1.txt"

data = data_path + data_name

# OUTPUT PATH
output_path = "/home/nlp/preprocess/outputs/"
output = output_path + data_name[:-4] + "-preprocessed.txt"

In [4]:
with open(data, "r") as f:
    data = f.read()

In [5]:
print(data[:1000])

그 결과가 무엇일지
잘들 봐둬 보여주겠어
8. 빼앗긴 나의 시간(rep.) - 멜레아강
[ 가사 / 접기 ]
날 이용하신 신이여
절대 이대로는 안돼
한낱 인간의 약속도
지켜져야 해
그게 운명
내 자리도 내 여자도 모두
어떤 것도 빼앗기지 않아
9. 불가능을 꿈꾸다 - 귀네비어
[ 가사 / 접기 ]
신이여 들리시나요
운명에 아픈 내 눈물이
나의 죄 이리 큰가요 두려워요
내가 원치 않는 결혼
나와는 상관없는 약속
지금 날 위로하는 건 이 노래뿐
이 노래뿐
(아~)
자유를 원해
(아~)
내 꿈 이뤄지길
신이여 보이시나요
절망에 빠진 내 슬픔이
어린 날 순수했던 꿈 사라지네
당신이 나에게 보낸
나를 흔드는 이 설레임
내가 기다린 한 사람, 이 사람뿐
이 사람뿐
(아~)
자유를 원해
(아~)
내 꿈 이뤄지길
(아~)
아~
(아~)
내 꿈 이뤄지길
10. 마법처럼 - 아더, 귀네비어 #
[ 가사 / 접기 ]
귀네비어
난 보여 우릴 막는 저 어둠의 그늘
떠도는 무성한 말 끝없이 날 가둬
아더
당신을 따라 시간 거슬러 바람 속을 헤매
고통에 맞서 악마와 싸워 이제 당신 앞에
아더, 귀네비어
우리를 이끌어준 이 마법처럼
그대 놓지 않으리 이 순간처럼
내 마음 가득 채워 지금 이 순간
자석처럼 끌린 우리 두 사람
귀네비어
별빛 속 우리 영혼 서롤 보고 있죠
성배를 향한 그 길 꿈 같은 시간들
아더
맘을 흐리는 어둠과 맞서 견딘 수많은 밤
언젠가 만날 당신을 위해 기도한 많은 날
아더, 귀네비어
우리를 이끌어준 이 마법처럼
그대 놓지 않으리 이 순간처럼
내 마음 가득 채워 지금 이 순간
자석 끌린 우리 두 사람
아더, 귀네비어
이제 더는 슬픔 오지 않게
나의 사랑 오직 그대만이
시린 운명 그대 함께라면 견딜 수 있어
아더
우리를 이끌어준 이 마법처럼
아더, 귀네비어
그대 놓지 않으리 이 순간처럼
내 마음 가득 채워 지금 이 순간
자석처럼 끌린 우리 두 사람
자석처럼 끌린 우리 두 사람
11. 새로운 시작 - 멜레아강 #
[ 가사 / 접기 ]
내가 꿈꾸던 세상의 시작
나의 힘으로 

In [6]:
lines = data.split("\n")
print(lines[:10])

['그 결과가 무엇일지', '잘들 봐둬 보여주겠어', '8. 빼앗긴 나의 시간(rep.) - 멜레아강', '[ 가사 / 접기 ]', '날 이용하신 신이여', '절대 이대로는 안돼', '한낱 인간의 약속도', '지켜져야 해', '그게 운명', '내 자리도 내 여자도 모두']


In [7]:
filtered_lines = []
trash_lines = []
trash_lines2 = []
trash_lines3 = []
trash_lines4 = []

sbracket_text = re.compile(' *\[[^\]]*\]')  # [], [주석], ...
bracket_text = re.compile(' *\([^)]*\)')  # (), (내용), ...
korean_text = re.compile('[가-힣]')
special_text = re.compile('[-!/\\#＃&＆*＊@＠§※☆★○●◎◇◆□■△▲▽▼→←↑↓↔〓◁◀▷▶♤♠♡♥♧♣⊙◈▣☞•]')
space_text = re.compile(' {2,}')
end_text = re.compile('[\.!?\'"#].{0,6}$')

for line in tqdm(lines):
    # remove contents with square brackets
    line = line.strip()
    line = sbracket_text.sub("", line)
    line = bracket_text.sub("", line)
    
    # write <|endoftext|> when the sentence is too short.
    if len(line) <= 80:
        line = "<|endoftext|>"
    elif len(line) <= 200 and special_text.match(line) != None:
        line = "<|endoftext|>"
    elif len(line) <= 200 and end_text.search(line) == None:
        line = "<|endoftext|>"
    elif korean_text.search(line) == None:
        line = "<|endoftext|>"

    
    # to prevent write <|endoftext|> twice
    if line == "<|endoftext|>":
        if len(filtered_lines) == 0:
            continue
        if filtered_lines[-1] == "<|endoftext|>":
            continue
        filtered_lines.append(line)
        continue
    

    # change the special characters to normal charcters
    # ㅁ
    line = line.replace("＃", "#")
    line = line.replace("＆", "&")
    line = line.replace("＊", "*")
    line = line.replace("＠", "@")
    # ㄴ
    line = line.replace("＂", "\"")
    line = line.replace("（", "(")
    line = line.replace("）", ")")
    line = line.replace("［", "[")
    line = line.replace("］", "]")
    line = line.replace("｛", "{")
    line = line.replace("｝", "}")
    line = line.replace("‘", "'")
    line = line.replace("’", "'")
    line = line.replace("“", "\"")
    line = line.replace("”", "\"")
    line = line.replace("〔", "[")
    line = line.replace("〕", "]")
    line = line.replace("〈", "<")
    line = line.replace("〉", ">")
    # ㄹ
    line = line.replace("％", "%")
    # ㄱ
    line = line.replace("　", " ")
    line = line.replace("！", "!")
    line = line.replace("＇", "'")
    line = line.replace("，", ",")
    line = line.replace("．", ".")
    line = line.replace("／", "/")
    line = line.replace("：", ":")
    line = line.replace("；", ";")
    line = line.replace("？", "?")
    line = line.replace("、", ",")
    line = line.replace("∼", "~")
    line = line.replace("´", "'")
    line = line.replace("～", "~")
    # ㄷ
    line = line.replace("＋", "+")
    line = line.replace("－", "-")
    line = line.replace("＜", "<")
    line = line.replace("＝", "=")
    line = line.replace("＞", ">")
    
    # edit link text not parsed well
    line = line.replace("[[", "")
    line = line.replace("]]", "")
    
    # edit special characters
    line = line.replace("&lt;", "<")
    line = line.replace("&gt;", ">")
    line = line.replace("&amp;", "&")
    line = line.replace("&quot;", "\"")

    # prevent more than 2 spaces
    line = space_text.sub(" ", line)

    filtered_lines.append(line)

100%|██████████| 8924667/8924667 [00:58<00:00, 152652.10it/s]


In [8]:
filtered_data = "\n".join(filtered_lines)
filtered_data = filtered_data.replace("\n<|endoftext|>", "<|endoftext|>")

In [9]:
with open(output, "w") as f:
    f.write(filtered_data)